In [1]:
import pandas as pd
from pathlib import Path
import json
import re
import numpy as np
import json
from llama_cpp import Llama, LlamaGrammar
import ast
from pprint import pformat
from omegaconf import OmegaConf

%load_ext autoreload
%autoreload 2

from src.utils import utils
from src.senti_rater import SentiRating
from src.local_llm import QwenLLM

In [3]:
llm = QwenLLM("./models/qwen/Qwen3-8B.Q4_K_M.gguf")
llm

AttributeError: 'QwenLLM' object has no attribute 'chat_format'

In [3]:
cfg = utils.load_config()
cfg.qwen_3_8b_q4_km.infer

{'n_ctx': 5000, 'script_path': '${path.script}', 'n_threads': 8, 'n_gpu_layers': 0, 'verbose': False, 'class_name': 'QwenLLM', 'chat_format': 'chatml', 'rope_freq_base': 1000000.0, 'temperature': 0.7, 'top_p': 0.8, 'top_k': 20, 'min_p': 0, 'max_tokens': 128, 'model_path': '${qwen.model_dir}/${qwen_3_8b_q4_km.filename}'}

In [13]:
a = pd.read_csv("./data/llama/llama_31_8b.csv")
a

,id,rating,reasons,infer_time
0,0.0,4.0,"['Siri delay is a minor setback, not a bear ca...",51.253482
1,1.0,3.0,"[""News does not directly mention Apple's perfo...",66.527472
2,2.0,3.0,"[""No direct mention of AAPL's performance or i...",58.628786
3,3.0,4.0,"[""Apple's potential for AI growth without sign...",63.216203
4,4.0,3.0,"[""Severance's success is neutral for AAPL stoc...",56.916073
...,...,...,...,...
236,236.0,3.0,"['Neutral news about aerospace industry, no di...",85.583081
237,237.0,3.0,['Neutral as the news focuses on analyst debat...,94.429291
238,238.0,5.0,"[""Boeing stole Lockheed's crown"", 'Potential c...",81.700443
239,239.0,3.0,"[""Neutral news about a potential deal involvin...",81.054414


In [11]:
a = a.drop(columns="Unnamed: 0")
a

,id,rating,reasons,infer_time
0,0.0,4.0,"['Siri delay is a minor setback, not a bear ca...",51.253482
1,1.0,3.0,"[""News does not directly mention Apple's perfo...",66.527472
2,2.0,3.0,"[""No direct mention of AAPL's performance or i...",58.628786
3,3.0,4.0,"[""Apple's potential for AI growth without sign...",63.216203
4,4.0,3.0,"[""Severance's success is neutral for AAPL stoc...",56.916073
...,...,...,...,...
238,238.0,5.0,"[""Boeing stole Lockheed's crown"", 'Potential c...",81.700443
239,239.0,3.0,"[""Neutral news about a potential deal involvin...",81.054414
240,240.0,3.0,"[""Neutral news about aerospace and defense sec...",87.738079
241,NaN,NaN,74.86568748899992,NaN


In [12]:
a.iloc[:-2, :].to_csv("./data/llama/llama_31_8b.csv", index=False)

In [30]:
cfg = utils.load_config()
cfg

{'gen_test_data': False, 'download': True, 'infer': True, 'model': 'llama_31_8b', 'gen_analysis': {'model_name': '${model}', 'path': '${path}', 'fig': '${fig}', 'ratings_path': '${${model}.ratings_path}', 'metrics_path': '${${model}.metrics_path}', 'req_cols': ['id', 'pub_date', 'ticker', 'title', 'content', 'rating', 'reasons'], 'start_id': 0, 'batch_size': 20}, 'std_infer': {'script_path': '${path.script}', 'n_threads': 8, 'n_gpu_layers': 0, 'verbose': False}, 'path': {'model_dir': './models', 'data_dir': './data', 'divergent': '${path.data_dir}/divergent.csv', 'test': '${path.data_dir}/test.csv', 'script': './src/local_llm.py'}, 'fig': {'theme': {'style': 'whitegrid', 'palette': 'pastel', 'font_scale': 1.5}, 'heatmap': {'annot': True, 'fmt': 'd', 'cmap': 'Blues', 'cbar': True}}, 'gemma': {'family': 'gemma', 'data_dir': '${path.data_dir}/${gemma.family}', 'model_dir': '${path.model_dir}/${gemma.family}', 'infer': {'class_name': 'LlamaLLM', 'n_ctx': 2048, 'temperature': 0.2}}, 'gemma_

In [33]:
cfg.gen_analysis.ratings_path

'./data/llama/ratings_llama_31_8b.csv'

In [29]:
OmegaConf.resolve(cfg.gemma_4b)

cfg.gemma_4b.infer


{'script_path': './src/local_llm.py', 'n_threads': 8, 'n_gpu_layers': 0, 'verbose': False, 'class_name': 'LlamaLLM', 'n_ctx': 2048, 'temperature': 0.2, 'model_path': './models/gemma/gemma-3-4b-it-q4_0.gguf'}

In [7]:
text = ('[\n'
    '\n'
    '{"id": 69, "rating": 4, "reasons": ["AI '
    'agents may reduce search usage", "GOOGL '
    'well-positioned for AI risks", '
    '"Subscription-based AI could benefit '
    'GOOGL"]}]')

text

'[\n\n{"id": 69, "rating": 4, "reasons": ["AI agents may reduce search usage", "GOOGL well-positioned for AI risks", "Subscription-based AI could benefit GOOGL"]}]'

In [13]:
utils.extract_dict_response(text)

'[{"id": 69, "rating": 4, "reasons": ["AI agents may reduce search usage", "GOOGL well-positioned for AI risks", "Subscription-based AI could benefit GOOGL"]}]'

In [4]:
ast.literal_eval(text)

[{'id': 69,
  'rating': 4,
  'reasons': ['AI agents may reduce search usage',
   'GOOGL well-positioned for AI risks',
   'Subscription-based AI could benefit GOOGL']}]

In [9]:
a = re.sub(r'\n+|\s\s+', "", text)
a = re.sub(r"```\w*", "", a)

print(a)

{'id': 69,'rating': 4,'reasons': ['AI agents may reduce traditional search usage, benefiting GOOGL.','GOOGL is seen as "best positioned" in the face of AI competition.','Subscriptions to AI agents imply less reliance on free search services.']}


In [12]:
a = a.replace('"', "'")
json.loads(utils.ensure_double_quotes(a))

{'id': 69,
 'rating': 4,
 'reasons': ['AI agents may reduce traditional search usage, benefiting GOOGL.',
  "GOOGL is seen as 'best positioned' in the face of AI competition.",
  'Subscriptions to AI agents imply less reliance on free search services.']}

In [102]:
a = utils.extract_dict_response(text)
a

'{"id": 4,"rating": 3,"reasons": ["Severance\'s success is neutral for AAPL stock, as it\'s specific to Apple TV+ services.", "Apple Intelligence delays and stretched valuation are neutral concerns, not directly impacting the news.", "China headwinds are a neutral factor, as the news doesn\'t specify their impact on AAPL stock."]}'

In [55]:
re.findall(r"```\w*", text)

['```', '```']

In [10]:
a = ('["Severance\'s success '
    "bolsters Apple's Services growth, a "
    'positive factor for AAPL," "Delays in '
    "Apple's Intelligence projects indicate "
    'minor operational hiccups," "China '
    'headwinds and valuation concerns may '
    'slightly weigh on AAPL\'s outlook."]')

a

'["Severance\'s success bolsters Apple\'s Services growth, a positive factor for AAPL," "Delays in Apple\'s Intelligence projects indicate minor operational hiccups," "China headwinds and valuation concerns may slightly weigh on AAPL\'s outlook."]'

In [11]:
ast.literal_eval(a)

["Severance's success bolsters Apple's Services growth, a positive factor for AAPL,Delays in Apple's Intelligence projects indicate minor operational hiccups,China headwinds and valuation concerns may slightly weigh on AAPL's outlook."]

In [4]:
import ast

a = ast.literal_eval(utils.extract_dict_response(text))
a

{'id': 4,
 'rating': 4,
 'reasons': ["Severance's success bolsters Apple's Services growth, a positive factor for AAPL,Delays in Apple's Intelligence projects indicate minor operational hiccups,China headwinds and valuation concerns may slightly weigh on AAPL's outlook."]}

In [8]:
print(a["reasons"][0])

Severance's success bolsters Apple's Services growth, a positive factor for AAPL,Delays in Apple's Intelligence projects indicate minor operational hiccups,China headwinds and valuation concerns may slightly weigh on AAPL's outlook.


In [6]:
len(a["reasons"])

1

In [102]:
a = re.findall(r'```\s*\{', text)
a

['```\n{']

In [78]:
text.find("```", 1000)

-1

In [108]:
def extract_dict_response(text: str) -> str:
    """Extract dictionary response from local LLM i.e. exclude thinking quotes."""

    # Get the starting message fence
    msg_fence = re.findall(r'```\w*', text)[0]

    # Get start and end index
    start_idx = text.find(msg_fence) + len(msg_fence)
    end_idx = -3 # Exclude ``` at the end

    # Extract dictionary response
    text = text[start_idx:end_idx]

    # Remove new lines and extra whitespaces
    return re.sub(r'\n+|\s\s+', "", text)

a = extract_dict_response(text)
a

'{"id": 81,"rating": 3,"reasons": ["Oracle\'s potential acquisition of TikTok\'s U.S. operations doesn\'t directly impact Meta\'s operations or financials.","The news focuses on Oracle, not Meta, leaving Meta\'s sentiment unchanged."]}'

In [1]:
import csv

with open('names.csv', 'w', newline='') as csvfile:
    # fieldnames = ['first_name', 'last_name']
    writer = csv.DictWriter(csvfile, fieldnames=['first_name', 'last_name'])

    writer.writeheader()
    writer.writerow({'first_name': 'Baked', 'last_name': ['Beans', 'one', 1]})
    writer.writerow({'first_name': 'Lovely', 'last_name': ['Spam']})
    writer.writerow({'first_name': 'Wonderful', 'last_name': ['Spam', "testing"]})

In [4]:
import csv
import time

with open("test.csv", "a", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["id", "value"])
    writer.writeheader()
    for i in range(5):
        writer.writerow({"id": i, "value": f"row {i}"})
        # f.flush()
        time.sleep(2)


In [81]:
text[664+7:-3]

'\n{\n  "id": 81,\n  "rating": 3,\n  "reasons": [\n    "Oracle\'s potential acquisition of TikTok\'s U.S. operations doesn\'t directly impact Meta\'s operations or financials.",\n    "The news focuses on Oracle, not Meta, leaving Meta\'s sentiment unchanged."\n  ]\n}\n'

In [70]:
df_list = [pd.read_csv(fpath) for fpath in Path("./data").rglob("metrics*.csv")]
df = pd.concat(df_list, axis=0).reset_index(drop=True)
df.insert(0, "model", [fpath.stem.split("_", maxsplit=1)[1] for fpath in Path("./data").rglob("metrics*.csv")])
df

,model,f1_score,precision,recall,total_time,mean_time,cmatrix
0,mistral_7b_q8,0.268466,0.324084,0.334244,0.0,0,"[[0, 1, 2, 6, 0], [1, 1, 27, 21, 2], [2, 0, 51..."
1,mistral_7b_q5_km,0.262865,0.226313,0.322385,0.0,0,"[[0, 1, 2, 6, 0], [1, 0, 31, 18, 2], [2, 0, 49..."
2,mistral_7b_q6,0.280831,0.334821,0.343656,0.0,0,"[[0, 1, 3, 5, 0], [1, 1, 26, 22, 2], [2, 0, 51..."
3,gemma_4b,0.521951,0.508197,0.604396,0.0,0,"[[5, 4, 0, 0, 0], [5, 33, 10, 4, 0], [5, 20, 5..."
4,gemma_12b,0.641296,0.712417,0.614897,0.0,0,"[[4, 4, 1, 0, 0], [1, 40, 6, 5, 0], [0, 18, 46..."
5,gemma_27b,0.638542,0.771315,0.599745,0.0,0,"[[4, 5, 0, 0, 0], [0, 40, 4, 8, 0], [0, 24, 45..."


In [67]:
for fpath in Path("./data").rglob("metrics*.csv"):
    print(fpath.stem)

metrics_mistral_7b_q8
metrics_mistral_7b_q5_km
metrics_mistral_7b_q6
metrics_gemma_4b
metrics_gemma_12b
metrics_gemma_27b


In [54]:
models = ["gemma_4b", "gemma_12b", "gemma_27b", "mistral_7b_q5_km", "mistral_7b_q6", "mistral_7b_q8"]

temp = []

for model in models:
    subfolder = "gemma" if "gemma" in model else "mistral"
    temp.append(pd.read_csv(f"./data/{subfolder}/metrics_{model}.csv"))

df = pd.concat(temp, axis=0).reset_index(drop=True)
df

,Unnamed: 0,f1_score,precision,recall,total_time,mean_time,cmatrix
0,0,0.521951,0.508197,0.604396,0.0,0,"[[5, 4, 0, 0, 0], [5, 33, 10, 4, 0], [5, 20, 5..."
1,0,0.641296,0.712417,0.614897,0.0,0,"[[4, 4, 1, 0, 0], [1, 40, 6, 5, 0], [0, 18, 46..."
2,0,0.638542,0.771315,0.599745,0.0,0,"[[4, 5, 0, 0, 0], [0, 40, 4, 8, 0], [0, 24, 45..."
3,0,0.262865,0.226313,0.322385,0.0,0,"[[0, 1, 2, 6, 0], [1, 0, 31, 18, 2], [2, 0, 49..."
4,0,0.280831,0.334821,0.343656,0.0,0,"[[0, 1, 3, 5, 0], [1, 1, 26, 22, 2], [2, 0, 51..."
5,0,0.268466,0.324084,0.334244,0.0,0,"[[0, 1, 2, 6, 0], [1, 1, 27, 21, 2], [2, 0, 51..."


In [56]:
df.insert(0, "model", models)
df

,model,f1_score,precision,recall,total_time,mean_time,cmatrix
0,gemma_4b,0.521951,0.508197,0.604396,0.0,0,"[[5, 4, 0, 0, 0], [5, 33, 10, 4, 0], [5, 20, 5..."
1,gemma_12b,0.641296,0.712417,0.614897,0.0,0,"[[4, 4, 1, 0, 0], [1, 40, 6, 5, 0], [0, 18, 46..."
2,gemma_27b,0.638542,0.771315,0.599745,0.0,0,"[[4, 5, 0, 0, 0], [0, 40, 4, 8, 0], [0, 24, 45..."
3,mistral_7b_q5_km,0.262865,0.226313,0.322385,0.0,0,"[[0, 1, 2, 6, 0], [1, 0, 31, 18, 2], [2, 0, 49..."
4,mistral_7b_q6,0.280831,0.334821,0.343656,0.0,0,"[[0, 1, 3, 5, 0], [1, 1, 26, 22, 2], [2, 0, 51..."
5,mistral_7b_q8,0.268466,0.324084,0.334244,0.0,0,"[[0, 1, 2, 6, 0], [1, 1, 27, 21, 2], [2, 0, 51..."


In [6]:
df = test.loc[
    :,
    [
        "ticker",
        "title",
        "content",
        "prosusai",
        "yiyanghkust",
        "ziweichen",
        "aventiq_ai",
        "rating",
        "reasons",
    ],
]
df.style.set_properties(
    subset=["title", "content", "reasons"],
    **{"width": "500px", "white-space": "normal"}
)

,ticker,title,content,prosusai,yiyanghkust,ziweichen,aventiq_ai,rating,reasons
0,AAPL,Apple Stock Nears Citi’s ‘Bear Case.’ Why It Remains a Buy Even After Siri Delay.,"Apple hit a stumbling block when its AI-enabled Siri update was delayed, but it can bounce back, say Citi analysts.",2,5,2,1,4,"[""Siri delay addressed by executive reshuffle"", ""Stock dip seen as buying opportunity""]"
1,AAPL,"Meet the Hard Asset That's Bigger Than Apple, Nvidia, and Microsoft Combined. One Wall Street Strategist Thinks It Could Hit a $40 Trillion Market Cap.","Shares of Apple, Nvidia, and Microsoft have exploded in recent years, as investors poured money into stocks that were best positioned to take advantage of the rise in artificial intelligence (AI). The U.S. government is also running a fiscal deficit, where spending during a fiscal year exceeds revenue.",1,5,1,1,3,"['No direct mention of Apple-specific developments', 'News focuses on comparative market cap analysis', 'U.S. fiscal deficit mentioned without Apple linkage']"
2,AAPL,1 Unstoppable Vanguard ETF to Confidently Buy With $400 During the Stock Market Sell-Off,"The S&P 500 (SNPINDEX: ^GSPC) index returned 25% (including dividends) in 2024, which was more than double its long-term average of 10.5%. The Vanguard Growth ETF (NYSEMKT: VUG) is an exchange-traded fund that tracks the performance of the CRSP Growth Index by holding the same stocks and maintaining similar weightings, and it has outperformed the S&P 500 every year, on average, since it was established in 2004. The Vanguard ETF is currently down 11% from its record high amid the broader stock market sell-off, so investors can pick up a single share for under $400.",1,5,3,1,3,"['VUG ETF news, no direct AAPL sentiment', 'Market sell-off impacts ETF, not AAPL-specific']"
3,AAPL,"Apple (AAPL) Poised for AI Boom Without Heavy GPU Investments, Says Evercore","We recently published a list of 10 High-Flying AI Stocks to Watch Today. In this article, we are going to take a look at where Apple Inc. (NASDAQ:AAPL) stands against other high-flying AI stocks to watch today. The most important conference in the tech world is in full swing, yet investors are skeptical about what’s […]",3,1,1,5,4,"['Positive outlook for AI-driven growth without capital-intensive GPU investments', 'Implied strategic advantage in cost-efficient AI infrastructure', 'Market perception of potential competitive edge in AI adoption']"
4,AAPL,Severance's Fame is Good for Apple TV+: Is it True for AAPL Stock?,"Severance's success bodes well for Apple's Services business. However, Apple Intelligence delays, stretched valuation, and China headwinds are concerns in 2025.",2,5,1,1,4,"['Services growth highlighted', 'Success offsets other concerns']"
5,AAPL,"Apple’s hearing aid software is impressive, but audiologists still matter",An expert from the Royal National Institute for Deaf People is enthusiastic about the innovation but cautions about its medical limitations.,2,5,3,1,4,"['Positive innovation in hearing health tech', 'Cautious note softens full bullish impact', 'Positioning AAPL in accessible healthcare']"
6,AAPL,Apple's $1 Billion Streaming Sinkhole: Is This the Costliest Gamble in Tech?,Apple TV+ is bleeding cash while rivals dominate--yet investors remain unfazed. Is Apple quietly rewriting the streaming playbook?,1,5,2,1,1,"['$1B annual losses from Apple TV+', ""Unprofitability compared to Apple's services"", 'Strategic gamble with unclear payoff']"
7,AAPL,Apple TV+ Inches to 45M Users as Losses Reach $1B Yearly,Apple TV+ is nearing 45 million subscribers despite incurring approximately $1 billion in losses each year,2,4,5,1,1,"['Annual losses reach $1B', 'Growth insufficient to offset costs']"
8,AAPL,Will Apple's Slow Artificial Intelligence Rollout Help or Hurt the Stock?,"Artificial intelligence (AI) is a huge opportunity for a device maker like Apple (NASDAQ: AAPL) to take advantage of. With more than 2.2 billion active Apple devices in the world, if th

In [14]:
new = pd.read_csv("./data/test_new.csv")
new

,id,pub_date,ticker,publisher,period,title,content,prosusai,yiyanghkust,ziweichen,aventiq_ai,rating,reasons
0,0,2025-04-01 20:43:00,AAPL,Barrons.com,2 hours ago,Apple Stock Nears Citi’s ‘Bear Case.’ Why It R...,Apple hit a stumbling block when its AI-enable...,2,5,2,1,4.0,"[""Siri delay addressed by executive reshuffle""..."
1,1,2025-04-01 20:43:00,AAPL,Motley Fool,2 hours ago,"Meet the Hard Asset That's Bigger Than Apple, ...","Shares of Apple, Nvidia, and Microsoft have ex...",1,5,1,1,3.0,"[""No direct mention of Apple-specific developm..."
2,2,2025-03-26 22:43:00,AAPL,Motley Fool,6 days ago,1 Unstoppable Vanguard ETF to Confidently Buy ...,The S&P 500 (SNPINDEX: ^GSPC) index returned 2...,1,5,3,1,3.0,"[""VUG ETF news, no direct AAPL sentiment"", ""Ma..."
3,3,2025-03-23 22:43:00,AAPL,Insider Monkey,9 days ago,Apple (AAPL) Poised for AI Boom Without Heavy ...,We recently published a list of 10 High-Flying...,3,1,1,5,4.0,"[""Positive outlook for AI-driven growth withou..."
4,4,2025-03-21 22:43:00,AAPL,Zacks,11 days ago,Severance's Fame is Good for Apple TV+: Is it ...,Severance's success bodes well for Apple's Ser...,2,5,1,1,4.0,"[""Services growth highlighted"", ""Success offse..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,248,2025-03-14 22:52:00,BA,Investor's Business Daily,18 days ago,"As The Dow Skids, Nvidia And This Aerospace Le...",The Dow Jones Industrial Average retreated 3.5...,1,5,5,1,5.0,"[""Boeing stock climbs against market trends"", ..."
249,249,2025-03-13 22:52:00,BA,Investing.com,19 days ago,Boeing stock: Market is undervaluing long-term...,Investing.com -- Citi analysts reaffirmed thei...,5,5,3,1,5.0,"[""Citi analysts reaffirmed a Buy rating."", ""Th..."
250,250,2025-03-13 22:52:00,BA,Quartz,19 days ago,Boeing could get hit hard by Trump's trade war...,President Donald Trump’s trade war could becom...,4,5,2,1,2.0,"[""Risk of tariffs impacting competitive positi..."
251,251,2025-03-12 22:52:00,BA,MT Newswires,20 days ago,"Boeing Reports Higher Deliveries, Lower Orders...",Boeing (BA) on Tuesday logged an increase in d...,2,5,3,1,4.0,"[""Boeing reported higher deliveries in Februar..."


In [16]:
response = utils.load_json("response.json")
response

{'id': 'chatcmpl-51f7d90b-e5a8-4749-bcbd-48c8bbe4a78a',
 'object': 'chat.completion',
 'created': 1746022245,
 'model': './models/gemma/gemma-3-4b-it-q4_0.gguf',
 'choices': [{'index': 0,
   'message': {'role': 'assistant',
    'content': '```json\n{\n  "id": 0,\n  "rating": 3,\n  "reasons": [\n    "Siri delay is a setback for Apple\'s AI strategy.",\n    "Citi analysts maintain a \'buy\' rating despite the delay."\n  ]\n}\n```'},
   'logprobs': None,
   'finish_reason': 'stop'}],
 'usage': {'prompt_tokens': 532, 'completion_tokens': 65, 'total_tokens': 597}}

In [24]:
a = response["choices"][0]["message"]["content"]
a

'```json\n{\n  "id": 0,\n  "rating": 3,\n  "reasons": [\n    "Siri delay is a setback for Apple\'s AI strategy.",\n    "Citi analysts maintain a \'buy\' rating despite the delay."\n  ]\n}\n```'

In [26]:
b = re.sub(r"\n\s*|```(json)*", "", a)
json.loads(b)

{'id': 0,
 'rating': 3,
 'reasons': ["Siri delay is a setback for Apple's AI strategy.",
  "Citi analysts maintain a 'buy' rating despite the delay."]}